# Lecture 1

## Problem 1: Know your Computer

Find out about the following for your computer:
1. ELF
2. Endian 
3. 64-bit?

<hr>

In base 10, we typically write numbers as $n = \sum_{p = 0}^{j} a_{p} \times 10^{p}$. For binary, we will have base 2, $n = \sum_{p = 0}^{j} a_{p} \times 2^{p}$. So, 2 $(1 \times 2^{1} + 1 \times 2^{0})$ will be 10, 4 $(1 \times 2^{2} + 0 \times 2^{1} + 0 \times 2^{0})$ will be 100, 8 will be 1000 and so on. 

Consider 8 bits, the first bit on the left (most significant bit or parity bit) is the sign bit $(-1)^{s}$, with $s$ being the state of the most significant bit. So, in the programme below, until 63 bits, we have the numbers until $2^{63}$ stored as positive (in binary, 1000000.....). However, once we reach the 64$^{\text{th}}$ bit, the state of the most significant bit becomes 1 instead of zero, so the sign changes and we get a negative number.

In [1]:
import numpy as np

In [2]:
j = np.int64(1)

In [3]:
n = 1
while n < 66:
    j *= 2
    print(n, j)
    n += 1

1 2
2 4
3 8
4 16
5 32
6 64
7 128
8 256
9 512
10 1024
11 2048
12 4096
13 8192
14 16384
15 32768
16 65536
17 131072
18 262144
19 524288
20 1048576
21 2097152
22 4194304
23 8388608
24 16777216
25 33554432
26 67108864
27 134217728
28 268435456
29 536870912
30 1073741824
31 2147483648
32 4294967296
33 8589934592
34 17179869184
35 34359738368
36 68719476736
37 137438953472
38 274877906944
39 549755813888
40 1099511627776
41 2199023255552
42 4398046511104
43 8796093022208
44 17592186044416
45 35184372088832
46 70368744177664
47 140737488355328
48 281474976710656
49 562949953421312
50 1125899906842624
51 2251799813685248
52 4503599627370496
53 9007199254740992
54 18014398509481984
55 36028797018963968
56 72057594037927936
57 144115188075855872
58 288230376151711744
59 576460752303423488
60 1152921504606846976
61 2305843009213693952
62 4611686018427387904
63 -9223372036854775808
64 0
65 0


/tmp/ipykernel_4567/2809919641.py:3: RuntimeWarning: overflow encountered in scalar multiply
  j *= 2


When we have 1000000 ($2^7$ in binary), multiplying by 2 pushes the 1 forward and gives us a minus sign. However, we do not get a 0 (and we'll see why). After that, we cannot push 1 any further as there are no bits left, so we end up with zeros

Fun fact: Zero has two binary representations -- 10000000 (-0) and 00000000 (+0) ("This is wasted space"). So instead, if the parity bit is 1, we first find the complement with respect to 1, i.e., flip $1 \to 0$ or $0 \to 1$ (except for the parity bit) and evaluate the number. Then we add 1 and our final integer is $(-1) \times (\text{number } + 1)$. This is exactly how negative numbers are read. They are flipped in order and what should have been -127 becomes -1. 

In [4]:
from sys import byteorder
print(byteorder)

little


Here, 'little' refers to little Endian. ***Endianness*** refers to the order in which bytes are arranged in memory. If one computer reads from left to right and another reads from right to left, there might be problems when the two computers want to communicate. Little Endian means that the computer stores the least significant bit first (first byte is the smallest) and goes from right to left. Big Endian, on the other hand, is the opposite.

## Problem 2: Base Conversion
Write a programme to convert a binary number to decimal and back. Generalise this for any base with the base as the input.

### Decimal to Binary Algorithm

1. **Initialise an empty list** `rem` to store the remainders.
2. **Loop to divide the decimal number** by 2 repeatedly:
    - Calculate the remainder when dividing by 2 (`dec % 2`), and append it to the `rem` list.
    - Update the decimal number by floor dividing by by 2 (`dec //= 2`).
3. **Reverse the remainder list** to get the binary digits in the correct order.
4. **Return the binary representation** by joining the elements of the reversed list as a string.

In [5]:
def dec_bin(dec):
    rem = []
    while dec > 0:
        remnum = dec % 2
        rem.append(remnum)
        dec //= 2
    bin = []
    while rem:
        bin.append(str(rem.pop()))
    return ''.join(bin)

print(dec_bin(87))

1010111


### Binary to Decimal Algorithm

1. **Initialise `bin_num` to 0** to store the result and `count` to 0 to keep track of the current binary digit's position.
2. **Loop through the binary number** (converted to a string):
    - Reverse the string to process from the least to the most significant bit.
    - For each digit, multiply it by $2^{\text{count}}$ (corresponding power of 2) and add the result to `bin_num`.
    - Increase `count` after each iteration to move to the next higher power of 2.
3. **Return the decimal result** stored in `bin_num`.

In [6]:
def bin_dec(binary):
    bin_num, count = 0, 0
    for i in str(binary)[::-1]:
        bin_num += int(i)*(2**count)
        count += 1
    return bin_num

print(bin_dec(1010111))

87


### Generalised for All Bases

The algorithm for this is roughly the same as the one for decimal to binary.

In [7]:
def base_conv(num, base):
    rem = []
    while num > 0:
        remnum = num % base
        rem.append(remnum)
        num //= base 
    new = []
    while rem:
        new.append(str(rem.pop()))
    return ''.join(new)

print(base_conv(87, 2))

1010111


### Some general rules for programmers:

1. Do not trust all the digits!
2. The smallest precision determines the overall precision. If a programme is truncating $\sqrt{2}$ to 1.414, everything else should only be taken to a precision of 3 digits after the decimal point.
3. Perform all computations in the highest precision mode and truncate only at the very end. 

### More Generalisation for Base Conversion: Converting Floating Point to Binary

In [8]:
def dec_to_binfloat(num):
    # index = []
    # integer = int(np.floor(np.abs(num))) # part before the decimal point
    # index = list(dec_bin(integer)) # binary version of integer part
    # index.append('.') # decimal point

    # if num >= 0:
    #     index.insert(0, '0')
    # elif num < 0:
    #     index.insert(0, '1') # parity bit

    # decpt = np.abs(num) - np.floor(np.abs(num)) # part after decimal point
    
    # for i in range(65 - len(index)): # entire thing is 64 bits including the separator
    #     if decpt >= 2 ** (-i): # if greater than that power of 2, update with 1 and subtract to update
    #         index.append('1') 
    #         decpt -= 2 ** (-i) 
    #     elif decpt < 2**-i: # if the power of 2 is larger, update with 0 
    #         index.append('0')
            
    # return ''.join(index)

    sign = '0' if num >= 0 else '1'
    num = np.abs(num)
    
    int_part = int(np.floor(num))
    int_bin = np.binary_repr(int_part)  # binary string version of integer part

    frac_part = num - int_part # part after decimal point
    frac_bin = []

    while frac_part and len(frac_bin) < 23:  # mantissa --> 23 bits
        frac_part *= 2
        bit = int(frac_part)
        frac_bin.append(str(bit))
        frac_part -= bit

    # scientific notation
    if int_bin != '0':  
        exp = len(int_bin) - 1  # shifting exponent
        mant = int_bin[1:] + ''.join(frac_bin)  # removing leading 1
    else:
        # when integer part is 0
        exp = -frac_bin.index('1') - 1
        mant = ''.join(frac_bin)[frac_bin.index('1') + 1:]

    exp += 127 # bias for single-precision
    exp_bin = np.binary_repr(exp, width = 8)  # converting exponent to 8 bits
    
    mant = mant.ljust(23, '0')[:23] # to ensure that the mantissa is just 23 bits
    
    binfloat = sign + exp_bin + mant
    
    return binfloat

In [9]:
print(dec_to_binfloat(-3.14))

11000000010010001111010111000010


### Converting a Decimal Floating Point to Any Base

In [10]:
def dec_any(num, base, prec = 10):
    if base < 2 or base > 36:
        print('Base must be between 2 and 36')

    sign = '-' if num < 0 else ''
    num = abs(num)

    # integer part
    int_part = int(num)
    int_str = ''
    if int_part == 0:
        int_str = '0'
    else:
        while int_part > 0:
            rem = int_part % base
            int_str = (str(rem) if rem < 10 else chr(55 + rem)) + int_str
            int_part //= base

    # fractional part
    frac_part = num - int(num)
    frac_str = ''
    while frac_part > 0 and len(frac_str) < prec:
        frac_part *= base
        digit = int(frac_part)
        frac_str += str(digit) if digit < 10 else chr(55 + digit)
        frac_part -= digit
        if frac_part == 0:
            break

    return sign + int_str + ('.' + frac_str if frac_str else '')

In [11]:
print(dec_any(-4.25, 2, 5)) # bin
print(dec_any(4.25, 8, 5))  # octal
print(dec_any(3.14, 16, 5)) # hexadecimal

-100.01
4.2
3.23D70
